## Notebook for testing code snippets

In [1]:
from datacube_stats.statistics import GeoMedian

In [3]:
import datacube_stats

In [ ]:
GeoMedian()

In [ ]:
im

## function to identify the amount of winter irrigation miss-identified as summer irrigation

In [ ]:
import numpy as np
import xarray as xr
import sys
sys.path.append('src')
import SpatialTools

In [ ]:
irrigated = 'results/nmdb/nmdb_Summer2013_14/nmdb_Summer2013_14_Irrigated_OEHandLS_masked.shp'
argmax = "/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/ndviArgMaxMin/mosaics/ndviArgMaxMin_20131101_mosaic.tif"

In [ ]:
def maxNDVIisinNovember(irrigated, argmax):
    timeofmax = xr.open_rasterio(argmax)#.isel(x=range(50000, 60000)).isel(y=range(50000,60000)).squeeze()
    print(timeofmax)
    timeofmax = timeofmax.isel(band = 0)
    print("--------------------------------")
    print(timeofmax)
    nov = timeofmax.where(timeofmax == 11)
    transform, projection = SpatialTools.geotransform(timeofmax, (timeofmax.x, timeofmax.y), epsg=3577)
    width,height = timeofmax.shape
    print("--------------------------------")
    print("rasterizing vector")
    mask = SpatialTools.rasterize_vector(irrigated, height, width,transform, projection, raster_path=None)
    result = nov.where(mask)
    print("--------------------------------")
    print('calculating counts')
    unique,counts=np.unique(result.values, return_counts=True)
    x = dict(zip(unique, counts))
    print(x)
    print("--------------------------------")
    print("area where max NDVI occurs in Novemeber is "  + str(x['11'] * (25*25) / 10000) + ' ha')

In [ ]:
maxNDVIisinNovember(irrigated, argmax)

## Simplifying polygons

In [ ]:
import sys
import os
import time
import copy
import geopandas as gpd

from shapely.geometry import asShape
from shapely.geometry import MultiLineString
from shapely.geometry import asLineString
from shapely.wkt import dumps
#from pprint import pprint

import shapefile
import numpy as np

sys.path.append('src/')
import bezier
import bendsimplify

In [ ]:
shape = "results/SICA/nmdb_Summer1998_99_Irrigated_OEHandLS_masked.shp"

In [ ]:
#nothihng seems to happen...
bendsimplify.bend_simplify("results/nmdb_Summer1998_99_Irrigated_OEHandLS_masked_objectID.shp")

In [ ]:
# This works but is not topologically aware so overlaps/slivers occur (if not
# for important analysis then could use overlay to remove overlaps)
gdf = gpd.read_file(shape)
gdf_simplified = gdf.copy()
gdf_simplified["geometry"] = gdf.geometry.simplify(tolerance=50,preserve_topology=True)
gdf.to_file("results/test_simplify_1998_99_tolerance50_shapely.shp")